<a href="https://colab.research.google.com/github/jena-shreyas/MLRC-papa/blob/main/PAPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [ ]:
!lsb_release -a
!python --version

In [2]:
%%shell
conda create --name env python=3.8 -y
source activate env
conda install pytorch -c pytorch-lts -c nvidia

python -m pip install -U pip
pip install --upgrade setuptools

pip install transformers==4.18.0

pip install datasets accelerate scipy scikit-learn

Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    setuptools-65.6.3          |     pyhd8ed1ab_0         619 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         761 KB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  bzip2              conda-forge/linux-64

In [3]:
%%shell
source activate env
git clone https://github.com/schwartz-lab-NLP/papa.git

Cloning into 'papa'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 120 (delta 40), reused 1 (delta 0), pack-reused 0
Receiving objects: 100% (120/120), 138.92 KiB | 6.31 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [ ]:
# %%shell
# source activate env
# pip uninstall transformers -y
# pip install transformers==4.18.0
# pip show transformers

In [4]:
# Copy local transformers content to installed transformers location
!cp -R papa/transformers/src/transformers/* /usr/local/envs/env/lib/python3.8/site-packages/transformers/

In [5]:
%%shell
source activate env
cd papa/transformers/papa_scripts

MODEL=bert-base-uncased
TASK=cola

python3 run_papa_glue_avgs_creator.py --model_name_or_path ${MODEL}  --task_name ${TASK}  --max_length 64    --per_device_train_batch_size 8   --output_dir ../../constant-matrices/ --cache_dir ../../cache/ --use_papa_preprocess true  --pad_to_max_length

01/07/2023 19:31:22 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no

Dataset glue downloaded and prepared to /content/papa/transformers/papa_scripts/../../cache/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.
100% 3/3 [00:00<00:00, 512.10it/s]
https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp2vlalt3c
Downloading: 100% 570/570 [00:00<00:00, 487kB/s]
storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
creating metadata file for /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab27

In [6]:

%%shell
source activate env
cd papa/transformers/papa_scripts

MODEL=bert-base-uncased
TASK=cola

python3 run_papa_glue.py --model_name_or_path ${MODEL} --task_name ${TASK} --do_eval --max_seq_length 64 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --output_dir ../../sorted-heads/ --cache_dir ../../cache/  --do_train --num_train_epochs 15.0 --learning_rate 2e-5 --lr_scheduler_type constant --disable_tqdm true --evaluation_strategy epoch --save_strategy no --use_papa_preprocess --use_freeze_extract_pooler true --static_heads_dir ../../constant-matrices/  --save_total_limit 0 --sort_calculating True

01/07/2023 19:38:59 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/07/2023 19:38:59 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=True,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=

In [7]:
%%shell
source activate env
cd papa/transformers/papa_scripts

MODEL=bert-base-uncased
TASK=cola

for static_heads_num in 0 72 126 135 144
 do
python3 run_papa_glue.py --model_name_or_path ${MODEL} --task_name ${TASK} --do_eval --max_seq_length 64 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --output_dir ../../output/ --overwrite_output_dir --cache_dir ../../cache/ --do_train --num_train_epochs 15.0 --learning_rate 2e-5 --lr_scheduler_type constant --disable_tqdm true --evaluation_strategy epoch --save_strategy no --use_papa_preprocess --grad_for_classifier_only true --use_freeze_extract_pooler true --static_heads_dir ../../constant-matrices/ --static_heads_num ${static_heads_num} --save_total_limit 0 --sorting_heads_dir ../../sorted-heads/
done

01/07/2023 20:02:05 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/07/2023 20:02:05 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=True,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=

In [ ]:
!ls papa/outputs

attention_patterns_sums.pt  attention_sums.pt  avgs.pt


In [ ]:
%%bash
source activate env
python

import torch
attention_sums = torch.load('papa/outputs/attention_sums.pt')
print(f"Shape of attention_sums : {attention_sums.size()}")         # (num_layers=12, num_heads=12, )

print(attention_sums[0][0].min())
# DIMENSIONS :

# num_layers = 12
# num_heads = 12
# num_input_tokens (n) = 64

Shape of attention_sums : torch.Size([12, 12, 64, 64])
tensor(0.)
